In [18]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [19]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta

# Reflect Tables into SQLAlchemy ORM

In [20]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [21]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [22]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [23]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [24]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [25]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [26]:
inspector = inspect(engine)

In [27]:

first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1d247d8a2e0>,
 'date': '2010-01-01',
 'id': 1,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65.0}

In [28]:
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1d247e5eca0>,
 'longitude': -157.8168,
 'station': 'USC00519397',
 'latitude': 21.2716,
 'elevation': 3.0,
 'id': 1,
 'name': 'WAIKIKI 717.2, HI US'}

In [29]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [30]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 

In [31]:
# Starting from the most recent data point in the database. 
most_recent_date = dt.date(2017,8,23)
# Calculate the date one year from the last date in data set.
last_date = most_recent_date - dt.timedelta(days=365)
print(last_date)
print(most_recent_date)

2016-08-23
2017-08-23


In [32]:
# Perform a query to retrieve the data and precipitation scores
precipitation_scores = session.query(Measurement.date,Measurement.prcp).\
    filter(Measurement.date > last_date).order_by(Measurement.date)

In [33]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(precipitation_scores, columns=['date','precipitation'])
df = df.set_index('date', inplace=True)

# Sort the dataframe by date
df.sort_values(['date'], ascending=True)

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [39]:
# Use Pandas Plotting with Matplotlib to plot the data
df_bar = df.plot(kind='bar', figsize=(10,8))
plt.xlabel('Date', rotation='vertical')
plt.ylabel('Inches')
plt.title('Precipitation Analysis')

AttributeError: 'NoneType' object has no attribute 'plot'

In [18]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [19]:
# Design a query to calculate the total number stations in the dataset


In [20]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [21]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [22]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [23]:
# Close Session
session.close()